In [1]:
## Imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.manifold import TSNE
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Ref: https://www.kaggle.com/code/rsuhara/ai-generated-text-detection-quick-baseline

In [2]:
## Read data
train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
test_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")


ai_generated_train_essays = pd.read_csv("/kaggle/input/llm-generated-essays/ai_generated_train_essays.csv")
ai_generated_train_essays_gpt4 = pd.read_csv("/kaggle/input/llm-generated-essays/ai_generated_train_essays_gpt-4.csv")


daigt_external_dataset_1 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_01.csv")
daigt_external_dataset_2 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_02.csv")
daigt_external_dataset_3 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_03.csv")
daigt_external_dataset_4 = pd.read_csv("/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv")
daigt_external_dataset = pd.concat([daigt_external_dataset_1, daigt_external_dataset_2, daigt_external_dataset_3, daigt_external_dataset_4])
daigt_external_dataset.drop(["source", "essay_id", "prompt"], axis=1, inplace=True)
daigt_external_dataset_train = daigt_external_dataset[daigt_external_dataset.fold != 0].drop("fold", axis=1)
daigt_external_dataset_val = daigt_external_dataset[daigt_external_dataset.fold == 0].drop("fold", axis=1)


train_essays = pd.concat([train_essays, ai_generated_train_essays, ai_generated_train_essays_gpt4])
train_essays['label'] = train_essays['generated']
train_essays.drop(["id", "prompt_id", "generated"], inplace=True, axis=1)
print(train_essays.shape)
train_essays.head()

(2078, 2)


,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0
2,"""America's love affair with it's vehicles seem...",0
3,How often do you ride in a car? Do you drive a...,0
4,Cars are a wonderful thing. They are perhaps o...,0


In [3]:
## Split the dataset
# train_essays, val_essays = train_test_split(train_essays, test_size=0.33)
## Merge with external dataset
# train_essays = pd.concat([daigt_external_dataset_train, train_essays])
# val_essays = pd.concat([daigt_external_dataset_val, val_essays])
train_essays = pd.concat([train_essays, daigt_external_dataset.drop('fold', axis=1)])
print("Train size", train_essays.shape)
# print("Val size", val_essays.shape)

Train size (161534, 2)


### Text Vectorization

In [4]:
df = pd.concat([train_essays['text'], test_essays['text']], axis=0)

vectorizer = TfidfVectorizer(stop_words='english', max_features=50000)
X = vectorizer.fit_transform(df)

### Word2Vec

ref: https://www.kaggle.com/code/nkitgupta/text-representations

In [5]:
Glove_path = "/kaggle/input/glove6b/glove.6B.300d.txt"

word2vec_output_file = 'glove.6B.300d.txt.word2vec'
glove2word2vec(Glove_path, word2vec_output_file)

# load the Stanford GloVe model
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

print("Glove Word Embeddings of word 'human' ")
glove_model['human']

/tmp/ipykernel_21/3006100231.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(Glove_path, word2vec_output_file)


Glove Word Embeddings of word 'human' 


array([ 1.6608e-01,  3.1670e-01, -5.8249e-01, -3.0978e-01, -4.3342e-02,
       -2.7902e-01, -2.2359e-01,  1.5822e-01,  1.4204e-01, -2.5546e+00,
        1.4570e-01, -2.4335e-01, -7.9048e-01, -7.0345e-02,  3.6125e-02,
        8.6590e-02,  7.9128e-01,  8.8366e-01, -3.8760e-01, -2.5363e-01,
       -2.1893e-01,  2.9416e-01, -1.0171e-01,  1.3125e-01,  1.7897e-01,
        5.8697e-01,  9.8880e-01,  1.3927e-01,  4.0796e-01,  1.0456e-02,
        8.6774e-03,  5.1483e-01, -1.1121e+00, -5.1464e-01,  9.6359e-02,
        1.8087e-01,  3.3565e-01,  2.4491e-01, -2.3970e-01, -3.6623e-01,
        5.1139e-01,  2.0225e-01, -9.9939e-02, -2.1025e-01, -4.4924e-01,
        2.7756e-01, -4.3422e-01,  2.9502e-01,  3.0119e-01,  7.0748e-02,
       -4.7604e-02,  5.4410e-01, -2.1035e-01,  9.3486e-02, -1.6013e-01,
       -6.2257e-02,  1.9005e-01, -4.4857e-01, -1.6407e-01, -2.2538e-01,
        1.6229e-01,  2.2353e-01,  1.4428e-01, -1.4320e-01, -5.4452e-01,
       -1.5810e-01,  1.6613e-01,  3.0517e-01,  2.2682e-01,  2.33

In [6]:
def plot_tsne(model, num):
    labels = []
    tokens = []
    for word in model.key_to_index:
        if word not in stop_words:
            tokens.append(np.array(model[word]))
            labels.append(word)
    tsne = TSNE(perplexity = 40, n_components = 2, init = 'pca', n_iter = 2500, random_state = 23)
    data = tsne.fit_transform(np.array(tokens[:num]))
    x = []
    y = []
    for each in data:
        x.append(each[0])
        y.append(each[1])
    plt.figure(figsize = (10, 10))
    for i in range(num):
        plt.scatter(x[i], y[i])
        plt.annotate(labels[i],
                     xy = (x[i], y[i]),
                     xytext = (5,2),
                     textcoords = 'offset points',
                     ha = 'right',
                     va = 'bottom')
    plt.show()

In [7]:
#visualising the cbow archtecture(only the first 300)
# plot_tsne(glove_model, 100)

In [8]:
# ref: https://www.kaggle.com/code/eswarbabu88/toxic-comment-glove-logistic-regression
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(glove_model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [9]:
df = pd.concat([train_essays['text'], test_essays['text']], axis=0)

X = np.array([sent2vec(sent) for sent in df])

### Logistic Regression

In [10]:
lr_model = LogisticRegression()
cv = StratifiedKFold(n_splits=5, shuffle=True)
auc_scores = []

# Split the data into training and validation for each fold
for train_idx, val_idx in cv.split(X[:train_essays.shape[0]], train_essays['label']):
    X_train, X_val = X[:train_essays.shape[0]][train_idx], X[:train_essays.shape[0]][val_idx]
    y_train, y_val = train_essays['label'].iloc[train_idx], train_essays['label'].iloc[val_idx]

    # Train the model on the training data
    lr_model.fit(X_train, y_train)
    
    # Predict probabilities for the positive class on the validation data
    preds_val_lr = lr_model.predict_proba(X_val)[:, 1]
    
    # Calculate ROC AUC score for the validation set
    auc_score = roc_auc_score(y_val, preds_val_lr)
    auc_scores.append(auc_score)

# Print the scores for each fold
for i, score in enumerate(auc_scores, 1):
    print(f'ROC AUC for fold {i}: {score:.4f}')

print('Average ROC AUC:', round(sum(auc_scores)/len(auc_scores), 4))
print('Standard deviation:', round((sum([(x - sum(auc_scores)/len(auc_scores))**2 for x in auc_scores])/len(auc_scores))**0.5, 4))

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

ROC AUC for fold 1: 0.9877
ROC AUC for fold 2: 0.9871
ROC AUC for fold 3: 0.9867
ROC AUC for fold 4: 0.9873
ROC AUC for fold 5: 0.9886
Average ROC AUC: 0.9875
Standard deviation: 0.0007


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### XGBoost

In [11]:
xgb_model = XGBClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True)
auc_scores = []

# Split the data into training and validation for each fold
for train_idx, val_idx in cv.split(X[:train_essays.shape[0]], train_essays['label']):
    X_train, X_val = X[:train_essays.shape[0]][train_idx], X[:train_essays.shape[0]][val_idx]
    y_train, y_val = train_essays['label'].iloc[train_idx], train_essays['label'].iloc[val_idx]

    # Train the model on the training data
    xgb_model.fit(X_train, y_train)
    
    # Predict probabilities for the positive class on the validation data
    preds_val_xgb = xgb_model.predict_proba(X_val)[:, 1]
    
    # Calculate ROC AUC score for the validation set
    auc_score = roc_auc_score(y_val, preds_val_xgb)
    auc_scores.append(auc_score)

# Print the scores for each fold
for i, score in enumerate(auc_scores, 1):
    print(f'ROC AUC for fold {i}: {score:.4f}')

print('Average ROC AUC:', round(sum(auc_scores)/len(auc_scores), 4))
print('Standard deviation:', round((sum([(x - sum(auc_scores)/len(auc_scores))**2 for x in auc_scores])/len(auc_scores))**0.5, 4))

ROC AUC for fold 1: 0.9998
ROC AUC for fold 2: 0.9998
ROC AUC for fold 3: 0.9999
ROC AUC for fold 4: 0.9996
ROC AUC for fold 5: 0.9999
Average ROC AUC: 0.9998
Standard deviation: 0.0001


In [12]:
# Create the ensemble model
ensemble = VotingClassifier(estimators=[('lr', lr_model), ('xgb', xgb_model)], voting='soft')

ensemble.fit(X_train, y_train)

# Predict on the validation set
y_pred = ensemble.predict(X_val)

# Print the classification report
print(classification_report(y_val, y_pred))

# Print the accuracy score
print(f'Accuracy: {roc_auc_score(y_val, y_pred)}\n')

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

           0       0.99      1.00      1.00     23349
           1       1.00      0.98      0.99      8957

    accuracy                           0.99     32306
   macro avg       1.00      0.99      0.99     32306
weighted avg       0.99      0.99      0.99     32306

Accuracy: 0.9911081465152366



In [13]:
preds_train = ensemble.predict_proba(X[:train_essays.shape[0]])[:,1]
preds_test = ensemble.predict_proba(X[train_essays.shape[0]:])[:,1]
print('ROC AUC train:', roc_auc_score(train_essays['label'], preds_train))

ROC AUC train: 0.9999210768985691


In [14]:
pd.DataFrame({'id':test_essays["id"],'generated':preds_test}).to_csv('submission.csv', index=False)

In [17]:
# def get_all_file_paths(directory):
  
#     # initializing empty file paths list
#     file_paths = []
  
#     # crawling through directory and subdirectories
#     for root, directories, files in os.walk(directory):
#         for filename in files:
#             # join the two strings in order to form the full filepath.
#             filepath = os.path.join(root, filename)
#             file_paths.append(filepath)
  
#     # returning all file paths
#     return file_paths        
  
# def main():
#     # path to folder which needs to be zipped
#     directory = '../input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'
  
#     # calling function to get all file paths in the directory
#     file_paths = get_all_file_paths(directory)

#     # writing files to a zipfile
#     with ZipFile('./lt.zip','w') as zip:
#         # writing each file one by one
#         for file in file_paths:
#             zip.write(file)
  
#     print('All files zipped successfully!')        
    
# main()


 
# zip_file = "./lt.zip"
 
# try:
#     with zipfile.ZipFile(zip_file) as z:
#         z.extractall()
#         print("Extracted all")
# except:
#     print("Invalid file")
    
# #move to cache
# !mv {'./input/language-tool-python-2-7-1/LanguageTool-5.7/LanguageTool-5.7'} {lt_path} 
# print(os.listdir('/root/.cache/language_tool_python/'))

# #remove files from output

# shutil.rmtree('./input')
# os.remove("./lt.zip")

In [18]:
# import language_tool_python
# correcter = language_tool_python.LanguageTool('en-US')

In [19]:
# import language_tool_python
# tool = language_tool_python.LanguageTool('en-US')
# def how_many_typos(text):    
#     return len(tool.check(text))

# train_essays['ntypos']=train_essays['text'].apply(lambda x: how_many_typos(x))
# train_essays['pred'] = -train_essays['ntypos']

In [20]:
# train_essays.head()